In [1]:
from memory import MemoryBuffer
from policies import PPOTrainer
from utils import gaussian_likelihood, reshape_train_var
import tensorflow as tf
from tensorflow.layers import dense, conv2d, max_pooling2d, flatten
import numpy as np
import time
import gym
from env import CartPoleEnv

In [2]:
class PPOTrainer():
    def __init__(self, in_op, out_op, value_out_op, act_type='discrete', sess=None, clip_val=0.2, ppo_iters=80,
                 target_kl=0.01, v_coef=1., entropy_coef=0.01):
        """
        Create a wrapper for RL networks for easy training.
        Args:
            in_op (tf.Placeholder): Observation input to architecture
            out_op (tf.Variable): Action output of architecture
            act_type (string): 'discrete' for a discrete actions space or 'continuous'
                               for a continuous actions space
            sess (tf.Session): A session if you would like to use a custom session,
                               if left none it will be automatically created
        """

        if not sess:
            self.renew_sess()
        
        self.in_op = in_op
        self.out_op = out_op
        self.value_out_op = value_out_op
        self._prev_weights = None
        self.clip_val = clip_val
        self.ppo_iters = ppo_iters
        self.target_kl = target_kl
        self.v_coef = v_coef
        self.entropy_coef = entropy_coef
        
        if act_type in ('discrete', 'd'):
            self.train = self._create_discrete_trainer()
            self.act_type = 'discrete'
        elif act_type in ('continuous', 'c'):
            self.train = self._create_continuous_trainer()
            self.act_type = 'continuous'
        else:
            raise TypeError('act_type must be \'discrete\' or \'continuous\'')
        
    def renew_sess(self):
        """
        Starts a new internal Tensorflow session
        """
        self.sess = tf.Session()
        
    def end_sess(self):
        """
        Ends the internal Tensorflow session if it exists
        """
        if self.sess:
            self.sess.close()
        
    def _create_discrete_trainer(self, optimizer=tf.train.AdamOptimizer()):
        """
        Creates a function for vanilla policy training with a discrete action space
        """
        # First passthrough
        
        self.act_holders = tf.placeholder(tf.int32, shape=[None])
        self.reward_holders = tf.placeholder(tf.float32, shape=[None])
        
        self.act_masks = tf.one_hot(self.act_holders, self.out_op.shape[1].value, dtype=tf.float32)
        self.resp_acts = tf.reduce_sum(self.act_masks *  self.out_op, axis=1)
        
        self.advantages = self.reward_holders - tf.squeeze(self.value_out_op)
        
        # Second passthrough
        
        self.advatange_holders = tf.placeholder(dtype=tf.float32, shape=self.advantages.shape)
        self.old_prob_holders = tf.placeholder(dtype=tf.float32, shape=self.resp_acts.shape)
 
        self.policy_ratio = self.resp_acts / self.old_prob_holders
        self.clipped_ratio = tf.clip_by_value(self.policy_ratio, 1 - self.clip_val, 1 + self.clip_val)

        self.min_loss = tf.minimum(self.policy_ratio * self.advatange_holders, self.clipped_ratio * self.advatange_holders)
        
        self.optimizer = tf.train.AdamOptimizer()

        # Actor update
        
        self.kl_divergence = tf.reduce_mean(tf.log(self.old_prob_holders) - tf.log(self.resp_acts))
        self.actor_loss = -tf.reduce_mean(self.min_loss)
        self.actor_update = self.optimizer.minimize(self.actor_loss)

        # Value update
        
        self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - tf.squeeze(self.value_out_op)))
        self.value_update = self.optimizer.minimize(self.value_loss)
        
        # Combined update
        
        self.entropy = -tf.reduce_mean(tf.reduce_sum(self.out_op * tf.log(1. / self.out_op), axis=1))
        self.combined_loss = self.actor_loss + self.v_coef * self.value_loss + self.entropy_coef * self.entropy
        self.combined_update = self.optimizer.minimize(self.combined_loss)
        
        def update_func(train_data):
            self.old_probs, self.old_advantages = self.sess.run([self.resp_acts, self.advantages], 
                                    feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                               self.act_holders: train_data[:, 1],
                                               self.reward_holders: train_data[:, 2]})
        
            for i in range(self.ppo_iters):
                kl_div, _ = self.sess.run([self.kl_divergence, self.combined_update], 
                               feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                    self.act_holders: reshape_train_var(train_data[:, 1]),
                                    self.reward_holders: train_data[:, 2],
                                    self.old_prob_holders: self.old_probs,
                                    self.advatange_holders: self.old_advantages})
                if kl_div > 1.5 * self.target_kl:
                    break

        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _create_continuous_trainer(self):
        """
        Creates a function for vanilla policy training with a continuous action space
        """
        # First passthrough
        
        self.act_holders = tf.placeholder(tf.float32, shape=[None, self.out_op.shape[1].value])
        self.reward_holders = tf.placeholder(tf.float32, shape=[None])
        
        self.std = tf.Variable(0.5 * np.ones(shape=self.out_op.shape[1].value), dtype=tf.float32)
        self.out_act = self.out_op + tf.random_normal(tf.shape(self.out_op), dtype=tf.float32) * self.std
        
        self.log_probs = gaussian_likelihood(self.act_holders, self.out_op, self.std)
        
        self.advantages = self.reward_holders - tf.squeeze(self.value_out_op)
        
        # Second passthrough
        
        self.advatange_holders = tf.placeholder(dtype=tf.float32, shape=self.advantages.shape)
        self.old_prob_holders = tf.placeholder(dtype=tf.float32, shape=self.log_probs.shape)
 
        self.policy_ratio = tf.exp(self.log_probs - self.old_prob_holders)
        self.clipped_ratio = tf.clip_by_value(self.policy_ratio, 1 - self.clip_val, 1 + self.clip_val)

        self.min_loss = tf.minimum(self.policy_ratio * self.advatange_holders, self.clipped_ratio * self.advatange_holders)
        
        self.optimizer = tf.train.AdamOptimizer()

        # Actor update
        
        self.kl_divergence = tf.reduce_mean(self.old_prob_holders - self.log_probs)
        self.actor_loss = -tf.reduce_mean(self.min_loss)
        self.actor_update = self.optimizer.minimize(self.actor_loss)

        # Value update
        
        self.value_loss = tf.reduce_mean(tf.square(self.reward_holders - tf.squeeze(self.value_out_op)))
        self.value_update = self.optimizer.minimize(self.value_loss)
        
        # Combined update
        
        self.entropy = -0.5 * tf.reduce_mean(tf.log(2 * np.pi * np.e * self.std))
        self.combined_loss = self.actor_loss + self.v_coef * self.value_loss + self.entropy_coef * self.entropy
        self.combined_update = self.optimizer.minimize(self.combined_loss)
        
        def update_func(train_data):
            self.old_probs, self.old_advantages = self.sess.run([self.log_probs, self.advantages], 
                                    feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                               self.act_holders: reshape_train_var(train_data[:, 1]),
                                               self.reward_holders: train_data[:, 2]})
            
            for i in range(self.ppo_iters):
                kl_div, _ = self.sess.run([self.kl_divergence, self.combined_update], 
                               feed_dict={self.in_op: reshape_train_var(train_data[:, 0]),
                                    self.act_holders: reshape_train_var(train_data[:, 1]),
                                    self.reward_holders: train_data[:, 2],
                                    self.old_prob_holders: self.old_probs,
                                    self.advatange_holders: self.old_advantages})
                if kl_div > 1.5 * self.target_kl:
                    break
            
            return kl_div, self.sess.run(self.entropy)

        self.sess.run(tf.global_variables_initializer())
        
        return update_func
        
    def _gen_discrete_act(self, obs):
        act_probs = self.sess.run(self.out_op, feed_dict={self.in_op: [obs]})
        act = np.random.choice(list(range(len(act_probs[0]))), p=act_probs[0])
        
        return act
    
    def _gen_continuous_act(self, obs):
        act_vect = self.sess.run(self.out_act, feed_dict={self.in_op: [obs]})[0]
        
        return np.array(act_vect)
        
    def gen_act(self, obs):
        if self.act_type == 'discrete':
            return self._gen_discrete_act(obs)
        else:
            return self._gen_continuous_act(obs)
        
    def train(self, obs, rewards, acts):
        raise RuntimeError('The train method was not properly created')

In [3]:
env = gym.make('CartPole-v1')

obs = tf.placeholder(tf.float32, shape=[None]+list(env.observation_space.shape))
dense1 = dense(obs, 32, activation=tf.tanh)
dense2 = dense(dense1, 32, activation=tf.tanh)
act_probs = dense(dense2, 2, activation=tf.nn.softmax)

v_dense1 = dense(obs, 32, activation=tf.tanh)
v_dense2 = dense(v_dense1, 32, activation=tf.tanh)
value = dense(v_dense2, 1)

network = PPOTrainer(obs, act_probs, value, act_type='d')

/home/ejmejm/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [4]:
# env = CartPoleEnv()
# # env = gym.make('BipedalWalker-v2')

# obs = tf.placeholder(tf.float32, shape=[None]+list(env.observation_space.shape))
# dense1 = dense(obs, 32, activation=tf.tanh)
# dense2 = dense(dense1, 32, activation=tf.tanh)
# act_probs = dense(dense2, env.action_space.shape[0])

# v_dense1 = dense(obs, 32, activation=tf.tanh)
# v_dense2 = dense(v_dense1, 32, activation=tf.tanh)
# value = dense(v_dense2, 1)

# network = PPOTrainer(obs, act_probs, value, act_type='c')

In [5]:
# env = gym.make('BipedalWalker-v2')

# obs = tf.placeholder(tf.float32, shape=[None]+list(env.observation_space.shape))
# dense1 = dense(obs, 32, activation=tf.tanh)
# dense2 = dense(dense1, 32, activation=tf.tanh)
# act_probs = dense(dense2, env.action_space.shape[0], activation=tf.tanh)

# v_dense1 = dense(obs, 32, activation=tf.tanh)
# v_dense2 = dense(v_dense1, 32, activation=tf.tanh)
# value = dense(v_dense2, 1)

# network = PPOTrainer(obs, act_probs, value, act_type='c', entropy_coef= 0.1)

In [6]:
n_episodes = 1000000
max_steps = 200
update_freq = 16
print_freq = 4

mb = MemoryBuffer()

In [7]:
all_rewards = []

for episode in range(n_episodes):
    ep_reward = 0
    
    mb.start_rollout()
    obs = env.reset()
    for step in range(max_steps):
        obs = obs.squeeze()
        act = network.gen_act(obs)
        
        obs = obs / 5.
        a_o = act
        act = np.clip(act, -1, 1)
        for i in obs:
            if abs(i) > 1:
                print('----------')
                print(i)
        
        obs_next, rew, d, _ = env.step(act)
        ep_reward += rew
        
        if False:
            env.render()
            time.sleep(0.02)
        
        mb.record(obs, act, rew)
        obs = obs_next
        
        if d:
            break
            
    all_rewards.append(ep_reward)
            
    if episode % update_freq == 0 and episode != 0:
        network.train(mb.to_data())
        
        if episode % (update_freq * print_freq) == 0:
            print(f'Update #{episode // update_freq}, Reward: {np.mean(all_rewards[-update_freq*print_freq:])}')
            # print(a_o)
            # print(ent)
            print()

Update #4, Reward: 12.671875

Update #8, Reward: 45.171875

Update #12, Reward: 166.34375

Update #16, Reward: 194.34375

Update #20, Reward: 200.0

Update #24, Reward: 200.0

Update #28, Reward: 200.0

Update #32, Reward: 199.953125



KeyboardInterrupt: 